In [4]:
# invisible(lapply(c(
#   "rentrez", "XML", "Biostrings", "tidyr"
# ), library, character.only = TRUE))
invisible(lapply(c(
  "tidyr", "Biostrings"
), library, character.only = TRUE))


# Set up appropriate contrasts for Type III ANOVAs
## First arg is for unordered factors, second is for ordered
## contr.sum contrast coefficients for each categorical add up to 0
## contr.poly contrasts measures the rate of change of the set of coefficients
options(contrasts = c("contr.sum", "contr.poly"))

ERROR: Error in FUN(X[[i]], ...): there is no package called ‘Biostrings’


# Reading the fasta files from GlobalAMFungi database

In [4]:
fasta_files <- lapply(
  list.files(
    path = "globalamfungi_LSU", pattern = "\\.fas$", full.names = TRUE
  ),
  \(x) readDNAStringSet(x)
)

# taxon <- readDNAStringSet(unzip("globalamfungi_LSU/acc_AY639235.zip"))
# 469163cb7da0bc34d45ed101ee1662a4|SampleID_8301|acc_AF396782|marker_LSU|abund_1_total_372
occurance_table <- data.frame()
for(taxon in fasta_files) {
  taxon_table <- setNames(as.data.frame(names(taxon)), "var") |>
    separate_wider_delim(
      var,
      delim = "|",
      names = c("uid", "sample_id", "taxon_id", "marker", "abundance")
    ) |>
    separate_wider_delim(
      abundance,
      delim = "_",
      names = c("_1", "abundance", "_2", "total")
    ) |>
    separate_wider_delim(
      sample_id,
      delim = "_",
      names = c("_3", "sample_id")
    ) |>
    separate_wider_delim(
      marker,
      delim = "_",
      names = c("_4", "marker")
    ) |>
    separate_wider_delim(
      taxon_id,
      delim = "_",
      names = c("_5", "taxon_id")
    ) |>
    (\(x) subset(x, select = !startsWith(colnames(x), "_")))()
  occurance_table <- rbind(occurance_table, taxon_table)
}
write.table(occurance_table, "occurances.tsv", row.names = FALSE, sep = "\t")

In [3]:
fasta_files <- lapply(
  list.files(
    path = "globalamfungi_species", pattern = "\\.fas$", full.names = TRUE
  ),
  \(x) readDNAStringSet(x)
)

# taxon <- readDNAStringSet(unzip("globalamfungi_LSU/acc_AY639235.zip"))
# 469163cb7da0bc34d45ed101ee1662a4|SampleID_8301|acc_AF396782|marker_LSU|abund_1_total_372
# occurance_table <- data.frame()
# for(taxon in fasta_files) {
#   taxon_table <- setNames(as.data.frame(names(taxon)), "var") |>
#     separate_wider_delim(
#       var,
#       delim = "|",
#       names = c("uid", "sample_id", "taxon_id", "marker", "abundance")
#     ) |>
#     separate_wider_delim(
#       abundance,
#       delim = "_",
#       names = c("_1", "abundance", "_2", "total")
#     ) |>
#     separate_wider_delim(
#       sample_id,
#       delim = "_",
#       names = c("_3", "sample_id")
#     ) |>
#     separate_wider_delim(
#       marker,
#       delim = "_",
#       names = c("_4", "marker")
#     ) |>
#     separate_wider_delim(
#       taxon_id,
#       delim = "_",
#       names = c("_5", "taxon_id")
#     ) |>
#     (\(x) subset(x, select = !startsWith(colnames(x), "_")))()
#   occurance_table <- rbind(occurance_table, taxon_table)
# }
# write.table(occurance_table, "occurances.tsv", row.names = FALSE, sep = "\t")

ERROR: Error in readDNAStringSet(x): could not find function "readDNAStringSet"


# Reading NCBI entries for the GlobalAMFungi LSU accession numbers

In [6]:
accessions <- unname(unlist(read.csv("accessions.csv", header = FALSE)))

xml_results <- entrez_fetch(
  db="nucleotide", id = accessions, rettype = "gbc",
  retmode = "xml", parsed = TRUE
)

fncols <- function(data, cname) {
  add <- cname[!cname %in% names(data)]
  if(length(add) != 0) data[add] <- NA
  data
}

# function to collate all publications associated with sequences
get_pub_info <- function(i){
  fetch2 <- entrez_fetch(
    db = "nucleotide", id = i,
    rettype = "gbc", retmode = "xml", parsed = TRUE
  )
  xml_list2 <- xmlToList(fetch2)
  ref_list <- xml_list2$INSDSeq$INSDSeq_references
  # extract publication fields info
  authors <- unlist(
    ref_list$INSDReference$INSDReference_authors
  ) |> paste(collapse = "; ")
  title <- ref_list$INSDReference$INSDReference_title
  journal <- ref_list$INSDReference$INSDReference_journal
  year <- gsub(".*\\((.*)\\).*", "\\1", journal)
  pm_id <- ref_list$INSDReference$INSDReference_pubmed
  remark <- ref_list$INSDReference$INSDReference_remark
  # create data frame of information
  pub.data <- data.frame(i, authors, journal, year)
  if(is.null(title) == FALSE) pub.data$title <- title
  if(is.null(pm_id) == FALSE) pub.data$pubmed_id <- pm_id
  if(is.null(remark) == FALSE) pub.data$remark <- remark
  pub.data <- fncols(pub.data, c("title", "pubmed_id", "remark"))
}

list_of_dfs <- lapply(accessions, get_pub_info)

df_combine <- do.call(rbind, list_of_dfs)
colnames(df_combine)[1] <- "NCBI_idv"
# df_combine <- tidyr::separate(
# 	df_combine, remark, c("text", "doi"), sep = "DOI:"
# ) # extract doi
# df_combine <- tidyr::separate(df_combine, doi, c("doi", "text2"), sep = ";")
# df_combine$remark <- paste(df_combine$text,df_combine$text2)
write.table(df_combine, "Pubs.tsv", sep = "\t", row.names = FALSE)